Use a RF to stack LSTM predictions with engineered features

In [15]:
import os
import logging

In [16]:
dir_path = os.path.realpath('..')

## Import data

In [17]:
import numpy as np
import pandas as pd

In [18]:
path = 'data/raw/train.csv'

full_path = os.path.join(dir_path, path)
df_train = pd.read_csv(full_path, header=0, index_col=0)
print("Dataset has {} rows, {} columns.".format(*df_train.shape))

Dataset has 95851 rows, 7 columns.


In [19]:
# fill NaN with string "unknown"
df_train.fillna('unknown',inplace=True)

## Pre-processing

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
split = round(len(df_train)/2)
df_A = df_train.iloc[:split-1]
df_B = df_train.iloc[split:]
print("Df A has {} rows, {} columns.".format(*df_A.shape))
print("Df B has {} rows, {} columns.".format(*df_B.shape))

Df A has 47925 rows, 7 columns.
Df B has 47925 rows, 7 columns.


In [22]:
dfs = [df_A, df_B]

## Model fit

In [23]:
from sklearn.model_selection import ParameterGrid
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from keras.callbacks import CSVLogger
from sklearn.metrics import log_loss

Using TensorFlow backend.


In [24]:
def save_model(model, model_path):
    # serialize model to JSON
    model_json = model.to_json()
    with open(model_path + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(model_path + ".h5")
    print("Saved model to disk")

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
import re

class TextCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, regex='\S+', remove_digits=False, english_only=False, stop_words=None, lower=True, filters=None):
        self.regex = regex
        self.remove_digits = remove_digits
        self.english_only = english_only
        self.stop_words = stop_words
        self.lower = lower
        self.filters = filters
        
    def transform(self, X, *args):
        tokenizer = RegexpTokenizer(self.regex)
        result = []
        for row in X:
            tokens = tokenizer.tokenize(row)
            if self.filters is not None:
                tokens = [re.sub(self.filters, '', t) for t in tokens]
            if self.lower:
                tokens = [t.lower() for t in tokens]
            if self.remove_digits:
                tokens = [t for t in tokens if not t.isdigit()]
            if self.english_only:
                english_words = set(nltk.corpus.words.words())
                tokens = [t for t in tokens if t in english_words]
            if self.stop_words is not None:
                tokens = [t for t in tokens if not t in self.stop_words]
            tokens = ' '.join(tokens)
            if tokens == '':
            	tokens = 'cleaned'
            result.append(tokens)
        return result
    
    def fit(self, *args):
        return self

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

class KerasProcesser(BaseEstimator, TransformerMixin):
    def __init__(self, num_words, maxlen):
        self.num_words = num_words
        self.maxlen = maxlen
        
    def transform(self, X, *args):
        tokenizer = Tokenizer(num_words=self.num_words)
        tokenizer.fit_on_texts(X)
        result = tokenizer.texts_to_sequences(X)
        result = pad_sequences(result, maxlen=self.maxlen, padding='post')
        return result, tokenizer, self.maxlen
    
    def fit(self, *args):
        return self

In [27]:
from keras.models import Sequential
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation

# Function to create model, required for KerasClassifier
def create_model():
    model = Sequential()
    model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length))
    model.add(Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)))
    model.add(GlobalMaxPool1D())
    model.add(Dense(50, activation="relu"))
    model.add(Dropout(0.1))
    model.add(Dense(6, activation='sigmoid'))  #multi-label (k-hot encoding)
    # summarize the model
    model.summary()
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [28]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/home/ec2-user/glove.840B.300d.txt', mode='rt', encoding='utf-8')
for line in f:
	values = line.split(' ')
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))



FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/glove.840B.300d.txt'

In [ ]:
p = Pipeline([
    ('cleaner', TextCleaner(remove_digits=False, english_only=False, stop_words=None)),
    ('keraser', KerasProcesser(num_words=20000, maxlen=65))
])

In [ ]:
%%time

i = 1

target = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
corpus = 'comment_text'

for df in dfs:
    model_name = 'LSTM_stack_' + str(i)
    print(df.shape)
    X = df[corpus]
    y = df[target]
    padded_train, t, max_length = p.transform(Xtrain)
    vocab_size = len(t.word_index) + 1
    
    # create a weight matrix for words in training docs
    embedding_matrix = np.zeros((vocab_size, 300))
    for word, i in t.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    
    # fit model
    model = KerasClassifier(build_fn=create_model, epochs=2, verbose=1)
    print(vocab_size, max_length)
    model.fit(padded_train, ytrain, verbose=1, validation_split=0.1, callbacks=[csv_logger])
    
    # save the model
    model_path = os.path.join(dir_path, 'models', model_name)
    save_model(model.model, model_path)
    
    i = i + 1

In [23]:
# make predictions





encoded_submission = t.texts_to_sequences(df_test[corpus])
padded_submission = pad_sequences(encoded_submission, maxlen=max_length, padding='post')
y_submission = model.model.predict(padded_submission, verbose=1)
submission = pd.DataFrame(y_submission, index=df_test.index, columns=target)
path = 'data/submissions/' + model_name + '.csv'
full_path = os.path.join(dir_path, path)
submission.to_csv(full_path, header=True, index=True)

226998/226998 [==============================] - 224s 988us/step
